In [ ]:
from flask import Flask, render_template, request
from datetime import date, datetime
import time
import re
import random
import json
from API import crawl_UTE as api

# convert accent vietnamese to no
def no_accent_vietnamese(s):
    s = s.lower()
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('đ', 'd', s)
    return s
# remove spacial and residual character 
def special_characters(str):
    str = re.sub(r"[^a-eg-ik-vxyA-EG-IK-VXY0-9 ]","",str)
    index = 0
    for i in str:
        if index != 0:
            if 'sxr'.find(i)>=0:
                if str[index-1] != ' ':
                    str =str.replace(i,"")
                    index-=1
        index+=1
    return str
# find a like b
def like(a,b):
    a = a.split(" ")
    for i in a:
        if b.find(i) < 0:
            return False
    return True
# check list a like b 
def check(a,b):
    for i in a:
        if like(i,b):
            return True
    return False
    
# get response 
def getRespone(userText,db):
    with open("data/"+db+".json",encoding='utf-8') as file:
        data = json.load(file)
    for intents in data["intents"]:
        for tag in intents["tag"]:
            if like(special_characters(no_accent_vietnamese(str(tag))),special_characters(no_accent_vietnamese(str(userText)))):
                for questions in intents['questions']:
                    if special_characters(no_accent_vietnamese(userText)).find(special_characters(no_accent_vietnamese(questions))) >=0:
                        return intents["answers"][random.randint(0, len(intents["answers"])-1)]
    for intents in data["intents"]:
        for questions in intents['questions']:
            if special_characters(no_accent_vietnamese(userText)).find(special_characters(no_accent_vietnamese(questions))) >=0:
                return intents["answers"][random.randint(0, len(intents["answers"])-1)]
    return "Thông tin này AI chưa được học :D"
# create app 
app = Flask(__name__)
# home 
@app.route("/")
def home():
    return render_template("index.html")
# get reply 
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    hour = ["mấy giờ","giờ hiện tại","bây giờ mấy giờ"]
    day = ["hôm nay ngày","ngày hiện tại","hôm ni ngày"]
    if check(hour,userText):
        return datetime.now().strftime("%H:%M:%S")
    else:
        if check(day,userText):
            return datetime.now().strftime("%d/%m/%Y")
        else:
            db = request.args.get('DB')
            if db == "":
                db = "InforGroup"
            return getRespone(userText,db)

# run main
if __name__ == "__main__":
    api.crawl_data()
    app.run()